In [1]:
import enum
import functools
import itertools
import numpy as np
import pandas as pd
from scipy import stats
from scipy import linalg
from typing import Callable, List, NamedTuple, Sequence, Tuple

BETA = np.array([0., 0.5], dtype=np.float64)
DESIGN_CLUSTERS = {
    'I': [[7, 10, 13, 16]],
    'II': [[7, 10, 13], [7, 10, 16], [7, 13, 16], [10, 13, 16]],
}
NUM_CLUSTERS = [15, 30, 60]
WITHIN_CLUSTER_CORRELATIONS = [0.5, 0.9]

CorrelationStructure = enum.Enum(
    'CorrelationStructure',
    'NONE EXCHANGEABLE EXPONENTIAL')

EstimationMethod = enum.Enum(
    'EstimationMethod',
    'GLS QL Sandwich')

In [2]:
class Experiment(NamedTuple('Experiment', [
    ('beta', np.array),
    ('error_variance', float),
    ('num_clusters', Sequence[Tuple[np.array, np.array]]),
    ('clusters', Sequence[np.array]),
    ('within_cluster_correlation', float),
    ('within_cluster_correlation_structure', CorrelationStructure),
])):
    """Encapsulates parameters for the data generating mechanism."""
    
    def sample_clusters(self) -> List[Tuple[np.array, np.array]]:
        return [self._sample_cluster() for _ in range(self.num_clusters)]
    
    def _sample_cluster(self) -> Tuple[np.array, np.array]:
        covariates = self._sample_cluster_covariates()
        covariates = np.column_stack((np.ones(len(covariates)), covariates))
        covariance = self._make_within_cluster_covariance(len(covariates))
        response = stats.multivariate_normal(
            mean=np.matmul(covariates, self.beta), cov=covariance).rvs()
        return covariates, response
    
    def _sample_cluster_covariates(self) -> np.array:
        return self.clusters[np.random.choice(len(self.clusters))]
    
    def _make_within_cluster_covariance(self, cluster_size):
        correlation = np.eye(cluster_size)
        if self.within_cluster_correlation_structure == CorrelationStructure.EXCHANGEABLE:
            correlation[correlation == 0] = self.within_cluster_correlation
        elif self.within_cluster_correlation_structure == CorrelationStructure.EXPONENTIAL:
            for i in range(cluster_size):
                for j in range(i + 1, cluster_size):
                    correlation[i, j] = correlation[j, i] = np.power(
                        self.within_cluster_correlation, np.abs(j - i))
        return self.error_variance*correlation
        
    @classmethod
    def from_template(
        cls,
        clusters,
        num_clusters,
        within_cluster_correlation,
        within_cluster_correlation_structure) -> 'Experiment':
        assert len(set([len(cluster) for cluster in clusters])) == 1,\
               'Clusters must be the same size.'
        
        return cls(beta=BETA,
                   clusters=clusters,
                   error_variance=1.,
                   num_clusters=num_clusters,
                   within_cluster_correlation=within_cluster_correlation,
                   within_cluster_correlation_structure=within_cluster_correlation_structure)

In [3]:
def sum_dict(acc, result):
    if type(acc) == dict:        
        return {key: sum_dict(value, result[key]) for key, value in acc.items()}   
    return acc + result
    
def divide_dict(results, d):
    if type(results) == dict:
        return {key: divide_dict(value, d) for key, value in results.items()}
    return results/d

In [4]:
def estimate_rho(epsilon_hat):
    covariance = np.outer(epsilon_hat, epsilon_hat)    
    rho_exchangeable = 0.
    rho_exponential = 0.
    for i in range(len(covariance)):
        for j in range(i + 1, len(covariance[i])):
            rho_exchangeable += covariance[i, j]
            if j - i == 1:
                rho_exponential += covariance[i, j]
    rho_exchangeable /= (np.square(covariance.shape[0]) - covariance.shape[0])/2
    rho_exponential /= covariance.shape[0] - 1
    return rho_exchangeable, rho_exponential

def make_correlation_matrices(clusters, beta_hat, sigma_2_hat):
    rho_exchangeable = 0.
    rho_exponential = 0.        
    for X, y in clusters:
        cluster_rho_exchangeable, cluster_rho_exponential = estimate_rho(
            (y - X.dot(beta_hat))/np.sqrt(sigma_2_hat))
        rho_exchangeable += cluster_rho_exchangeable
        rho_exponential += cluster_rho_exponential
            
    rho_exchangeable /= len(clusters)
    rho_exponential /= len(clusters)
    
    correlation_matrices = []
    for X, y in clusters:
        exchangeable_matrix = np.eye(len(y))
        exchangeable_matrix[exchangeable_matrix == 0] = rho_exchangeable
        exponential_matrix = np.eye(len(y))
        for i in range(len(y) - 1):
            for j in range(i + 1, len(y)):
                exponential_matrix[i, j] = exponential_matrix[j, i] = np.power(rho_exponential, j - i)
        correlation_matrices.append({
            CorrelationStructure.NONE.name: np.eye(len(y)),
            CorrelationStructure.EXCHANGEABLE.name: exchangeable_matrix,
            CorrelationStructure.EXPONENTIAL.name: exponential_matrix,            
        })
        
    return correlation_matrices

def estimate_beta_hats(clusters, correlation_matrices):
    def estimate_beta_hat(X, y, correlation_matrix):
        weight = linalg.cho_solve(
            linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix)))
        gram_matrix = linalg.cho_factor(X.T.dot(weight).dot(X))
        return linalg.cho_solve(gram_matrix, X.T.dot(weight).dot(y))
    
    beta_hats = [
        {
            key: estimate_beta_hat(X, y, inv_weight)
            for key, inv_weight in inv_weights.items()
        } for (X, y), inv_weights in zip(clusters, correlation_matrices)
    ]
    return divide_dict(functools.reduce(sum_dict, beta_hats), len(beta_hats))

def estimate_covariance(clusters,
                        correlation_matrices,
                        method,
                        beta_hat):
    if method != EstimationMethod.Sandwich:
        covariance = np.zeros((len(beta_hat), len(beta_hat)))
        dispersion_factor = 0.
        total = 0.
        for (X, y), correlation_matrix in zip(clusters, correlation_matrices):
            weight = linalg.cho_solve(
                linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix)))
            covariance += X.T.dot(weight).dot(X)
            dispersion_factor += np.sum(np.square(y - X.dot(beta_hat)))
            total += len(y)
        covariance = linalg.cho_solve(linalg.cho_factor(covariance), np.eye(len(beta_hat)))
        dispersion_factor /= total - len(beta_hat)
        return covariance if method == EstimationMethod.GLS else covariance*dispersion_factor
    # Sandwich estimation.
    bread = np.zeros((len(beta_hat), len(beta_hat)))
    meat = np.zeros((len(beta_hat), len(beta_hat)))
    for (X, y), correlation_matrix in zip(clusters, correlation_matrices):
        weight = linalg.cho_solve(
                linalg.cho_factor(correlation_matrix), np.eye(len(correlation_matrix)))
        bread += X.T.dot(weight).dot(X)        
        epsilon_hat = y - X.dot(beta_hat)
        meat += X.T.dot(weight).dot(np.outer(epsilon_hat, epsilon_hat)).dot(weight).dot(X)        
    bread = linalg.cho_solve(linalg.cho_factor(bread), np.eye(len(beta_hat)))    
    return bread.dot(meat).dot(bread)
    
def run_experiment(experiment, estimate_beta=False):
    clusters = experiment.sample_clusters()
    X = np.vstack([X for X, _ in clusters])
    y = np.hstack([y for _, y in clusters])
    
    gram_matrix_ols = X.T.dot(X)
    
    beta_hat_ols = linalg.cho_solve(linalg.cho_factor(gram_matrix_ols), X.T.dot(y))
    sigma_2_hat_ols = np.sum(np.square(y - X.dot(beta_hat_ols)))/(len(y) - len(beta_hat_ols))
    
    correlation_matrices = make_correlation_matrices(
        clusters,
        beta_hat_ols,
        sigma_2_hat_ols)
    beta_hats = estimate_beta_hats(clusters, correlation_matrices)
    
    if estimate_beta:
        return beta_hats
    
    return {
        method.name: {
            correlation_structure: np.sqrt(estimate_covariance(
                clusters,
                [matrix_dict[correlation_structure] for matrix_dict in correlation_matrices],
                method,
                beta_hat)[1, 1])
            for correlation_structure, beta_hat in beta_hats.items()
        }
        for method in EstimationMethod
    }

def run_experiments(experiment, num_trials):                    
    results = [run_experiment(experiment) for _ in range(num_trials)]
    results = functools.reduce(sum_dict, results)
    return divide_dict(results, num_trials)

In [5]:
experiments = [
    Experiment.from_template(design, num_clusters, correlation_coefficient, correlation_structure)
    for design, num_clusters, correlation_coefficient, correlation_structure
    in
    itertools.product(
       [DESIGN_CLUSTERS['I'], DESIGN_CLUSTERS['II']],
       NUM_CLUSTERS,
       WITHIN_CLUSTER_CORRELATIONS,
       [CorrelationStructure.EXCHANGEABLE, CorrelationStructure.EXPONENTIAL])
]

In [6]:
def index_experiment(experiment):    
    return (experiment.num_clusters,
            [k for k, v in DESIGN_CLUSTERS.items() if experiment.clusters == v][0],
            experiment.within_cluster_correlation_structure.name,
            experiment.within_cluster_correlation)

simulation_results = pd.DataFrame(
    index=pd.MultiIndex.from_product(
        [NUM_CLUSTERS, DESIGN_CLUSTERS.keys(),
         [CorrelationStructure.EXCHANGEABLE.name, CorrelationStructure.EXPONENTIAL.name],
         WITHIN_CLUSTER_CORRELATIONS,         
        ],
        names=['$n$', 'Design', 'Correlation structure', 'Correlation']),
    columns=pd.MultiIndex.from_product(
        [[value.name for value in EstimationMethod],
         [value.name for value in CorrelationStructure]],
        names=['Estimator', 'Assumed correlation']
    ))

In [7]:
for experiment in experiments:
    simulation_results.loc[index_experiment(experiment)] = (
        pd.DataFrame.from_dict(run_experiments(experiment, 512), orient='index').stack())
simulation_results

Estimator                                           GLS               \
Assumed correlation                                NONE EXCHANGEABLE   
$n$ Design Correlation structure Correlation                           
15  I      EXCHANGEABLE          0.5            0.03849    0.0282542   
                                 0.9            0.03849    0.0146767   
           EXPONENTIAL           0.5            0.03849     0.031532   
                                 0.9            0.03849    0.0173594   
    II     EXCHANGEABLE          0.5          0.0447061    0.0345911   
                                 0.9          0.0446511    0.0189873   
           EXPONENTIAL           0.5          0.0446385    0.0364608   
                                 0.9          0.0446972    0.0204039   
30  I      EXCHANGEABLE          0.5          0.0272166    0.0194407   
                                 0.9          0.0272166   0.00947367   
           EXPONENTIAL           0.5          0.0272166    0.0221318   
                                 0.9          0.0272166    0.0113306   
    II     EXCHANGEABLE          0.5          0.0315007    0.0239796   
                                 0.9          0.0314915    0.0120761   
           EXPONENTIAL           0.5          0.0314717    0.0254677   
                                 0.9          0.0315838    0.0132227   
60  I      EXCHANGEABLE          0.5           0.019245    0.0137959   
                                 0.9           0.019245   0.00642283   
           EXPONENTIAL           0.5           0.019245    0.0155561   
                                 0.9           0.019245   0.00792887   
    II     EXCHANGEABLE          0.5          0.0222621    0.0166359   
                                 0.9          0.0222389   0.00799746   
           EXPONENTIAL           0.5          0.0222351    0.0177563   
                                 0.9          0.0222251   0.00890048   

Estimator                                                        QL  \
Assumed correlation                          EXPONENTIAL       NONE   
$n$ Design Correlation structure Correlation                          
15  I      EXCHANGEABLE          0.5           0.0377314  0.0380706   
                                 0.9           0.0246676  0.0373303   
           EXPONENTIAL           0.5            0.037926  0.0381073   
                                 0.9           0.0244534  0.0373148   
    II     EXCHANGEABLE          0.5           0.0404487  0.0439269   
                                 0.9           0.0249984  0.0430464   
           EXPONENTIAL           0.5           0.0402742  0.0440887   
                                 0.9            0.024675   0.044021   
30  I      EXCHANGEABLE          0.5           0.0266391  0.0271901   
                                 0.9           0.0161044   0.026773   
           EXPONENTIAL           0.5           0.0268398  0.0270468   
                                 0.9           0.0159261  0.0270948   
    II     EXCHANGEABLE          0.5           0.0282966  0.0310985   
                                 0.9           0.0160047  0.0309143   
           EXPONENTIAL           0.5           0.0281977  0.0312558   
                                 0.9            0.015885  0.0314301   
60  I      EXCHANGEABLE          0.5           0.0189527  0.0191568   
                                 0.9           0.0110612  0.0189807   
           EXPONENTIAL           0.5           0.0189426   0.019211   
                                 0.9           0.0109883  0.0190884   
    II     EXCHANGEABLE          0.5           0.0198433   0.022228   
                                 0.9           0.0107784  0.0220079   
           EXPONENTIAL           0.5           0.0197935  0.0221206   
                                 0.9           0.0106574  0.0220756   

Estimator                                                              \
Assumed correlation                          EXCHANGEABLE EXPONE

In [25]:
import os

if not os.path.isdir('simulation_results'):
    os.mkdir('simulation_results')

for key, values in simulation_results.iterrows():
    file_name = '-'.join(map(str, key)).replace('.', '_')
    with open('simulation_results/{}.tex'.format(file_name), 'w') as f:
        f.write(' & '.join(map(lambda v: str(np.round(v, decimals=5)), values.values)))

In [23]:
print(simulation_results.to_latex(header=False, index=False))

\begin{tabular}{lllllllll}
\toprule
   0.03849 &    0.028166 &  0.0377848 &   0.037981 &   0.0275438 &    0.03722 &   0.0254299 &   0.0254299 &   0.0258039 \\
   0.03849 &   0.0146097 &   0.024612 &  0.0373179 &   0.0138226 &   0.023376 &   0.0116229 &   0.0116229 &   0.0122187 \\
   0.03849 &   0.0315155 &  0.0379207 &  0.0382045 &   0.0311561 &  0.0375915 &   0.0368714 &   0.0368714 &   0.0364408 \\
   0.03849 &   0.0173828 &  0.0248282 &  0.0371405 &   0.0163858 &  0.0235084 &   0.0200754 &   0.0200754 &   0.0199016 \\
 0.0446102 &   0.0339637 &  0.0396971 &   0.044506 &   0.0335858 &   0.039402 &   0.0346374 &   0.0311796 &   0.0315873 \\
 0.0447161 &   0.0188928 &  0.0249425 &  0.0440977 &   0.0181755 &   0.024096 &   0.0260229 &   0.0141971 &   0.0146531 \\
 0.0446853 &    0.036738 &  0.0404423 &  0.0437766 &   0.0357462 &  0.0394319 &   0.0393327 &   0.0374427 &   0.0374033 \\
 0.0447774 &   0.0204913 &  0.0247859 &   0.044248 &   0.0198952 &  0.0240859 &   0.0295767 &   0.01980

In [18]:
experiment = Experiment.from_template(
    DESIGN_CLUSTERS['I'],
    NUM_CLUSTERS[2],
    WITHIN_CLUSTER_CORRELATIONS[1],
    CorrelationStructure.EXPONENTIAL)

beta_hats = [
    run_experiment(
        experiment, estimate_beta=True)[CorrelationStructure.EXCHANGEABLE.name]
    for _ in range(64)
]
np.sqrt(np.var(beta_hats, ddof=1, axis=0))

array([0.18292895, 0.01059142])

In [ ]:
simulation_results.loc[index_experiment(experiment)] = (
    pd.DataFrame.from_dict(tmp, orient='index').stack())
simulation_results.to_dict()

<CorrelationStructure.NONE: 1>

In [177]:
np.random.choice([[7, 9], [2, 3], [1, 2]])

ValueError: a must be 1-dimensional

In [42]:
linalg.cholesky(tmp).T.dot(linalg.cholesky(tmp))

array([[1. , 0.5, 0.5, 0.5],
       [0.5, 1. , 0.5, 0.5],
       [0.5, 0.5, 1. , 0.5],
       [0.5, 0.5, 0.5, 1. ]])

In [55]:
linalg.inv(tmp)

array([[ 1.6, -0.4, -0.4, -0.4],
       [-0.4,  1.6, -0.4, -0.4],
       [-0.4, -0.4,  1.6, -0.4],
       [-0.4, -0.4, -0.4,  1.6]])

In [52]:
np.sqrt(3)/6

0.28867513459481287